### 数据预处理

在数据清理上，还有一个更好用的 Pandas 库，我们会在后期的教学当中，一步步深入到。

可以看到不同的 array 创建方式，可能会有不同的 dtype，而之前的 data.dtype=object，说明Python list 直接转换过来的 data 是无法参与诸多 Numpy 计算的。 而只有 dtype 为 int，float 这样的数值形式，才能参与运算。

那怎么办呢？我们怎么才能把那个又有 string，又有 None，还有数值的数据转换为 Numpy 能运算的数据呢？ 方法很简单，我们筛选过滤一下。

In [1]:
raw_data = [
["Name", "StudentID", "Age", "AttendClass", "Score"],
["小明", 20131, 10, 1, 67],
["小花", 20132, 11, 1, 88],
["小菜", 20133, None, 1, "98"],
["小七", 20134, 8, 1, 110],
["花菜", 20134, 98, 0, None],
["刘欣", 20136, 12, 0, 12]
]
print(raw_data)


[['Name', 'StudentID', 'Age', 'AttendClass', 'Score'], ['小明', 20131, 10, 1, 67], ['小花', 20132, 11, 1, 88], ['小菜', 20133, None, 1, '98'], ['小七', 20134, 8, 1, 110], ['花菜', 20134, 98, 0, None], ['刘欣', 20136, 12, 0, 12]]


In [2]:
import numpy as np
data = np.array(raw_data)
data

array([['Name', 'StudentID', 'Age', 'AttendClass', 'Score'],
       ['小明', 20131, 10, 1, 67],
       ['小花', 20132, 11, 1, 88],
       ['小菜', 20133, None, 1, '98'],
       ['小七', 20134, 8, 1, 110],
       ['花菜', 20134, 98, 0, None],
       ['刘欣', 20136, 12, 0, 12]], dtype=object)

In [3]:
# 通过循环的方法得到float类型再转化为numpy数组 不要直接转化为numpy数组
data_process = []
for i in range(len(raw_data)):
    if i == 0:
        continue    # 不要首行字符串
    # 去掉首列名字
    data_process.append(raw_data[i][1:])  
    
data_process
data = np.array(data_process, dtype=np.float64)
print("data.dtype", data.dtype)
print(data) 

data.dtype float64
[[2.0131e+04 1.0000e+01 1.0000e+00 6.7000e+01]
 [2.0132e+04 1.1000e+01 1.0000e+00 8.8000e+01]
 [2.0133e+04        nan 1.0000e+00 9.8000e+01]
 [2.0134e+04 8.0000e+00 1.0000e+00 1.1000e+02]
 [2.0134e+04 9.8000e+01 0.0000e+00        nan]
 [2.0136e+04 1.2000e+01 0.0000e+00 1.2000e+01]]


### 清洗数据

我们想要看看有没有什么数据是不合逻辑的

In [4]:
#我发现学号有重复，可能是在输入学生信息的时候手误输错了

sid = data[:, 0]
unique, counts = np.unique(sid, return_counts=True)

#来查找数据中独一无二的数据的,返回的是一个数组，数组的第一个元素是独一无二的数据，第二个元素是每个独一无二的数据出现的次数

print(counts)

data[4, 0] = 20135 
print(data)

[1 1 1 2 1]
[[2.0131e+04 1.0000e+01 1.0000e+00 6.7000e+01]
 [2.0132e+04 1.1000e+01 1.0000e+00 8.8000e+01]
 [2.0133e+04        nan 1.0000e+00 9.8000e+01]
 [2.0134e+04 8.0000e+00 1.0000e+00 1.1000e+02]
 [2.0135e+04 9.8000e+01 0.0000e+00        nan]
 [2.0136e+04 1.2000e+01 0.0000e+00 1.2000e+01]]


我们来看看第二列 Age 年龄。一眼可以发现，存在两个问题，有一个同学没有年龄。我们看看能不能用平均年龄补全这个缺失的信息

In [5]:
is_nan = np.isnan(data[:,1])
print("is_nan:", is_nan)
nan_idx = np.argwhere(is_nan)

# 计算有数据的平均年龄，用 ~ 符号可以 True/False 对调
mean_age = data[~np.isnan(data[:,1]), 1].mean()
print("有数据的平均年龄：", mean_age)

# 所以我觉得我也得把这个 98 岁的当异常数据看待。（在你的项目中，你也要综合判断哪些是异常数据）

# ~ 表示 True/False 对调，& 就是逐个做 Python and 的运算
normal_age_mask = ~np.isnan(data[:,1]) & (data[:,1] < 20)
print("normal_age_mask:", normal_age_mask)

normal_age_mean = data[normal_age_mask, 1].mean()
print("normal_age_mean:", normal_age_mean)

data[~normal_age_mask, 1] = normal_age_mean
print("ages:", data[:, 1])

is_nan: [False False  True False False False]
有数据的平均年龄： 27.8
normal_age_mask: [ True  True False  True False  True]
normal_age_mean: 10.25
ages: [10.   11.   10.25  8.   10.25 12.  ]


因为没上课，就没成绩，但是倒数第一行，没上课，怎么还有成绩？还有倒数第三行，成绩居然超出了满分 100 分。这些情况都是我们要处理的情况

In [6]:
# 没上课的转成 nan
data[data[:,2] == 0, 3] = np.nan

# 超过 100 分和低于 0 分的都处理一下
data[:, 3] = np.clip(data[:, 3], 0, 100)   # 当你使用 == 操作符时，NumPy会自动处理类型转换，以便进行比较操作。

print(data[:, 2:])

[[  1.  67.]
 [  1.  88.]
 [  1.  98.]
 [  1. 100.]
 [  0.  nan]
 [  0.  nan]]
